# Qual a acertividade das previsões publicadas pelo BACEN?

O Banco Central publica semanalmente um boletim contendo as estatísticas calculadas considerando as expectativas de mercado para os principais índices econômicos brasileiros, que contém valores para o ano corrente e os próximos, chamado Relatório Focus.

Os valores apontados no relatório tem como origem diversos agentes econômicos que não fazem parte deste órgão, como instituições financeiras, empresas e universidades.

Estas previsões são impactadas por fatores externos de baixa ou nenhuma previsibilidade como a pandemia de COVID-19, guerra na Ucrânia, resultados de eleições, entre outros, que causam impacto em seus números.

Mesmo assim, é válido entender quão acertivas são estas expectativas de mercado, já que elas ajudam a nortear os planejamentos financeiros/estratégicos de diversas empresas e pessoas, nos mais variados ramos de atuação.



# Fonte de dados

## Sistema Expectativas de Mercado

Para captura das previsões de mercado, o BACEN disponibiliza uma plataforma (https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/aplicacao#!/recursos) que possibilita montar consultas a serem feitas em sua API apenas selecionando o peíodo desejado, e posteriormente os parâmetros.

O endereço base da API é ``https ://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/odata/[codigo_recurso]?$format=json&[Outros Parâmetros]``, considerando os códigos de recursos abaixo.

> ExpectativaMercadoMensais, ExpectativasMercadoTrimestrais, ExpectativasMercadoAnuais, ExpectativasMercadoInflacao12Meses, ExpectativasMercadoTop5Mensais, ExpectativasMercadoTop5Anuais, ExpectativasMercadoInstituicoes

Então, basta usar o link obtido para efetuar a consulta e analisar os dados.

In [1]:
from IPython.display import IFrame
IFrame(
    'https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/aplicacao#!/recursos', 
    width='100%', 
    height=250
)

In [2]:
IFrame(
    'https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/aplicacao#!' \
    '/recursos/ExpectativasMercadoAnuais#eyJmb3JtdWxhcmlvIjp7IiRmb3JtYXQiOiJqc29uIiwiJHRvcCI6MTAwfX0=',
    width='100%',
    height=250
)

## Histórico dos índices

Situação similar ocorre para os dados reais, onde existe uma API chamada BCData/SGS - Sistema Gerenciador de Séries Temporais, conforme exemplo a seguir.

As consultas são feitas utilizando o endereço padrão ``http://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo_serie}/dados?formato=json&dataInicial={dataInicial}&dataFinal={dataFinal}``.

In [3]:
IFrame(
    'https://dadosabertos.bcb.gov.br/dataset/24363-indice-de-atividade-economica-do-banco-central' \
    '---ibc-br/resource/7d5869ad-b479-47d3-b98f-b99d8d008d5b',
    width='100%',
    height=250
)

Para encontrar o código da série deseja existem dois métodos, diretamente na plataforma do BACEN [``https://www3.bcb.gov.br/sgspub/localizarseries/localizarSeries.do?method=prepararTelaLocalizarSeries``](https://www3.bcb.gov.br/sgspub/localizarseries/localizarSeries.do?method=prepararTelaLocalizarSeries), ou utilizando a biblioteca `pybacen`.

Na plataforma do BACEN, basta inserir o nome do indicador no campo "Pesquisa textual", avaliar qual indicador mais se adequa a necessidade e usar seu código na API.

<img src="img/pesquisa_indicador_sgs.png" alt="Sistema Gerenciador de Séries Temporais" style="width: 100%;"/>

Na biblioteca ``pybacen`` a pesquisa se torna um pouco mais acertiva, porque é possível pesquisar partes do nome do indicador, bem como escolher a unidade, periodicidade e fonte.

O resultado da pesquisa é um DataFrame do pandas, e nem sempre o nome do indicador (nm_serie) aparecerá completo. Para resolver isto, é possível tirar o limitador de caracteres exibidos no DataFrame (o padrão é 50) utilizando o método ``pd.options.display.max_colwidth = None``.

Despois de realizar a pesquisa, é interessante voltar para o padrão, pra evitar DataFrames muito grandes e de difícil entendimento.

Outro ponto a considerar é o aviso que é exibido a cada pesquisa, informando para confirmar a informação diretamente no site do BACEN.

In [4]:
import pandas as pd
pd.options.display.max_colwidth = None

In [5]:
from pybacen.bacen import time_series

time_series.read_bacen_code(search_text='%ipca%12 meses', period='M')


c:\python\lib\site-packages\pybacen\bacen\time_series.py:116: UserWarning:

Check the website: https://www3.bcb.gov.br/sgspub/localizarseries/localizarSeries.do?method=prepararTelaLocalizarSeries



,bacen_code,nm_serie,unit,periodicity,source,special
11820,13522,Índice nacional de preços ao consumidor - amplo (IPCA) - em 12 meses,%,M,IBGE,N


In [6]:
time_series.read_bacen_code(search_text='%meta%selic%')

c:\python\lib\site-packages\pybacen\bacen\time_series.py:116: UserWarning:

Check the website: https://www3.bcb.gov.br/sgspub/localizarseries/localizarSeries.do?method=prepararTelaLocalizarSeries



,bacen_code,nm_serie,unit,periodicity,source,special
405,432,Taxa de juros - Meta Selic definida pelo Copom,% a.a.,D,Copom,N


In [7]:
pd.options.display.max_colwidth = 50

# Captura dos dados e construção das tabelas/DataFrames

Para elaborar